In [1]:

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, TrainerCallback
from peft import LoraConfig, TaskType, get_peft_model
import torch
import psutil
import numpy as np



2025-06-09 18:13:41.593449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749492821.774848      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749492821.828390      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 8.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidi

In [3]:
torch.cuda.set_per_process_memory_fraction(1.0, 0)  # Use maximum available memory
torch.cuda.memory_max_split_size_mb = 64  # Set the max split size to avoid fragmentation

In [4]:
def print_memory_footprint():
    # GPU memory usage
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.memory_allocated() / (1024 ** 3)  # Convert to GB
        gpu_memory_cached = torch.cuda.memory_reserved() / (1024 ** 3)  # Cached memory
        print(f"[GPU] Memory Allocated: {gpu_memory:.2f} GB, Cached: {gpu_memory_cached:.2f} GB")
    else:
        print("[GPU] No GPU detected.")

    # CPU memory usage
    memory = psutil.virtual_memory()
    used_memory_gb = memory.used / (1024 ** 3)  # Convert to GB
    total_memory_gb = memory.total / (1024 ** 3)
    print(f"[CPU] Memory Usage: {used_memory_gb:.2f} GB / {total_memory_gb:.2f} GB")

In [5]:
'''
print("First example of blended_skill_talk training set:")
print(dataset['train'][0])
'''

'\nprint("First example of blended_skill_talk training set:")\nprint(dataset[\'train\'][0])\n'

In [6]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token, so we use eos_token

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [7]:
# Load dataset and tokenizer
dataset = load_dataset("allenai/soda")



def tokenize_function(examples):
    # Concatenate dialog turns into a single string for language modeling
    texts = [" ".join(dialog) for dialog in examples["dialogue"]]
    return tokenizer(texts, truncation=True, max_length=512)

# Tokenize datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'])
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500))

README.md:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/689M [00:00<?, ?B/s]

valid.parquet:   0%|          | 0.00/82.9M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/84.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1191582 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/146346 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/148968 [00:00<?, ? examples/s]

Map:   0%|          | 0/1191582 [00:00<?, ? examples/s]

Map:   0%|          | 0/146346 [00:00<?, ? examples/s]

Map:   0%|          | 0/148968 [00:00<?, ? examples/s]

In [8]:
print(dataset)
tokenized_datasets


DatasetDict({
    train: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 1191582
    })
    validation: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 146346
    })
    test: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 148968
    })
})


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1191582
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 146346
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 148968
    })
})

In [9]:
# LoRA configuration for causal language modeling
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=2,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj"],
)

# %%

In [10]:
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained GPT-2 language model
# Load pre-trained GPT-2 model
#model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(device)
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                                             torch_dtype=torch.bfloat16,).to(device)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Enable gradient checkpointing if you run into memory issues
#model.gradient_checkpointing_enable()

# %%
# Print the model's architecture to inspect the names of the modules
#print(model)

# %%
print_memory_footprint()

Using device: cuda


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

[GPU] Memory Allocated: 2.05 GB, Cached: 2.14 GB
[CPU] Memory Usage: 2.38 GB / 31.35 GB


In [11]:
# %%

# Data collator for language modeling (masks tokens for prediction)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) # mlm=False for causal LM

# Perplexity as metric
#perplexity_metric = evaluate.load("perplexity", module_type="metric")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    shift_logits = torch.tensor(logits[:, :-1, :])
    shift_labels = torch.tensor(labels[:, 1:])
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    ppl = torch.exp(loss).item()
    return {"perplexity": ppl}

# Training args
training_args = TrainingArguments(
    output_dir="./lama-dialog-finetuned",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=10,
    #evaluation_strategy="epoch",
    #save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False,
    bf16=True,  # Enable bfloat16
    fp16=False,  # Disable fp16 to avoid conflicts
)
# %%
print_memory_footprint()

[GPU] Memory Allocated: 2.05 GB, Cached: 2.14 GB
[CPU] Memory Usage: 2.39 GB / 31.35 GB


In [12]:
class MemoryCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        print("\nMemory footprint after evaluation:")
        print_memory_footprint()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    #eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
    callbacks=[MemoryCallback()]
)


/tmp/ipykernel_35/3187942852.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [13]:

print("--- TinyLlama Before Fine-tuning ---")
model_before_finetune = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
             torch_dtype=torch.bfloat16).to(device)

input_text_before = "Hello, how are you?"
input_ids_before = tokenizer.encode(input_text_before, return_tensors="pt").to(device)

print(f"Input: {input_text_before}")
output_before = model_before_finetune.generate(
    input_ids_before,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
generated_response_before = tokenizer.decode(output_before[0], skip_special_tokens=True)
print(f"Generated Response (Before): {generated_response_before}")
print("---")


--- TinyLlama Before Fine-tuning ---


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Input: Hello, how are you?
Generated Response (Before): Hello, how are you?
I am fine, thank you.
How are you? I am fine, thank you.
Can you repeat that again?
Can you repeat that again, please?
Can you repeat that again,
---


In [14]:

print("--- TinyLlama Before Fine-tuning ---")
#model_before_finetune = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

input_text_before = "The USA celebrate independence day on "
input_ids_before = tokenizer.encode(input_text_before, return_tensors="pt").to(device)

print(f"Input: {input_text_before}")
output_before = model_before_finetune.generate(
    input_ids_before,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
generated_response_before = tokenizer.decode(output_before[0], skip_special_tokens=True)
print(f"Generated Response (Before): {generated_response_before}")
print("---")


--- TinyLlama Before Fine-tuning ---
Input: The USA celebrate independence day on 
Generated Response (Before): The USA celebrate independence day on 4th July.

2. The USA celebrate independence day on 4th July.

3. The USA celebrate independence day on 4th July.

4. The
---


In [15]:
any(p.requires_grad for p in model.parameters())

True

In [16]:
trainer.train()

Step,Training Loss
10,0.784100
20,0.763500
30,0.938800
40,0.844500
50,0.828700


TrainOutput(global_step=50, training_loss=0.8319242572784424, metrics={'train_runtime': 26.3385, 'train_samples_per_second': 3.797, 'train_steps_per_second': 1.898, 'total_flos': 135250878947328.0, 'train_loss': 0.8319242572784424, 'epoch': 1.0})

In [17]:
trainer.save_model('TinyLlama-new-1000')

In [18]:
print_memory_footprint()

[GPU] Memory Allocated: 4.12 GB, Cached: 7.52 GB
[CPU] Memory Usage: 3.14 GB / 31.35 GB


In [19]:
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# For generation, we'll load the model and use the `generate` method
model_for_generation = AutoModelForCausalLM.from_pretrained('TinyLlama-new-1000').to(device)
#model_for_generation = get_peft_model(model_for_generation, lora_config) # Ensure LoRA is applied

input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")

print_memory_footprint()

Using device: cuda

Generating response:


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Input: Hello, how are you?
Generated Response: Hello, how are you?
I am fine, thank you.
How are you? I am fine, thank you.
Can you repeat that again?
Can you repeat that again, please?
Can you repeat that again, please?
Can you repeat that again, please?
Can you repeat that again, please?
Can you repeat that again, please?
Can you repeat that again, please?
Can you repeat that again, please?
Can you repeat that again, please?
Can you repeat that again, please?
Can you repeat that again, please?
Can you repeat that again, please?
Can you repeat that again, please?
Can you repeat that again, please
[GPU] Memory Allocated: 8.22 GB, Cached: 10.54 GB
[CPU] Memory Usage: 3.93 GB / 31.35 GB


In [20]:
input_text = "The USA celebrate independence day on "
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")


Generating response:
Input: The USA celebrate independence day on 
Generated Response: The USA celebrate independence day on 4th July.

4. The USA celebrate independence day on 4th July.

5. The USA celebrate independence day on 4th July.

6. The USA celebrate independence day on 4th July.

7. The USA celebrate independence day on 4th July.

8. The USA celebrate independence day on 4th July.

9. The USA celebrate independence day on 4th July.

10. The USA celebrate independence day on 4th July.

11. The USA celebrate independence day on 4th July.

12. The USA


In [21]:
input_text = "So light is made up of colors?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")


Generating response:
Input: So light is made up of colors?
Generated Response: So light is made up of colors?


In [22]:
small_eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 500
})

In [23]:
from nltk.translate.bleu_score import sentence_bleu


# Tokenize the generated response
candidate = generated_response.split()

# Define reference responses (tokenized)
references = [
    "Yes, that's right. Light is made up of different colors, and when those colors mix together, we see white light. But light can also bend and bounce off surfaces. How does it do that? When light hits a surface, it can either reflect off of the surface or refract through the surface. Reflection is when the light bounces off of the surface, and refraction is when the light bends as it passes through the surface. So when light reflects off of a mirror, it's because the light is bouncing off of the surface of the mirror? That's right. Mirrors are very good at reflecting light because they have a smooth, flat surface. And when light refracts through something like a prism, it's because the light is bending as it passes through the surface of the prism? That's right. When light passes through a prism, it bends because the Prism bends the light."
]

# Tokenize each reference
reference_tokens = [ref.split() for ref in references]

# Calculate BLEU scores
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4  # Helps with zero scores for higher n-grams

print('Bleu 4: %f' % sentence_bleu(reference_tokens, candidate, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie))



Bleu 4: 0.000000


In [24]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
smoothie = SmoothingFunction().method4

input_texts = [
    "So light is made up of colors?",
    "Why is the sky blue?",
    "How does gravity work?",
    "What causes rainbows?",
    "Why do we see lightning before thunder?",
    "How do plants make food?",
    "Why do we sleep?",
    "What is photosynthesis?",
    "How does the internet work?",
    "Why do birds migrate?"
]

reference_answers = [
    "Yes, light is made up of different colors. When they combine, they appear white. This can be separated with prisms, for example.",
    "The sky appears blue due to a phenomenon called Rayleigh scattering. Shorter blue wavelengths scatter more than red ones.",
    "Gravity is a force that attracts objects with mass toward each other. Earth pulls objects towards its center due to gravity.",
    "Rainbows occur when sunlight is both refracted and reflected inside raindrops, separating the light into different colors.",
    "Light travels faster than sound, so we see lightning before we hear thunder.",
    "Plants make food through photosynthesis, using sunlight, water, and carbon dioxide to produce energy and oxygen.",
    "We sleep to allow the body and brain to rest, recover, and process information from the day.",
    "Photosynthesis is the process plants use to convert sunlight into energy using chlorophyll, water, and carbon dioxide.",
    "The internet is a global network of computers that communicate via protocols like TCP/IP to exchange data.",
    "Birds migrate to find better food sources, breeding grounds, or climates that are more suitable during different seasons."
]

bleu_scores = []

for i in range(10):
    input_text = input_texts[i]
    reference = reference_answers[i]

    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    output = model_for_generation.generate(
        input_ids,
        max_length=150,
        num_return_sequences=1,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    candidate = generated_response.split()
    reference_tokens = [reference.split()]

    score = sentence_bleu(reference_tokens, candidate, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    bleu_scores.append(score)

    print(f"\nPrompt {i+1}: {input_text}")
    print(f"Generated: {generated_response}")
    print(f"Reference: {reference}")
    print(f"BLEU-4 Score: {score:.4f}")

print("\nAverage BLEU-4 Score across all prompts: %.4f" % (sum(bleu_scores) / len(bleu_scores)))



Prompt 1: So light is made up of colors?
Generated: So light is made up of colors?
Reference: Yes, light is made up of different colors. When they combine, they appear white. This can be separated with prisms, for example.
BLEU-4 Score: 0.0721

Prompt 2: Why is the sky blue?
Generated: Why is the sky blue?

1. The sun is the primary source of light in the sky.

2. The Earth is tilted on its axis, which causes the sun to appear to rise in the east and set in the west.

3. The Earth's rotation around the sun causes the seasons to occur.

4. The Earth's rotation causes the tilt of the Earth's axis, which causes the sun to appear to move across the sky.

5. The Earth's rotation causes the Earth's axis to tilt towards the sun, which causes the seasons to occur.

6. The Earth's rotation causes the Earth's axis to t
Reference: The sky appears blue due to a phenomenon called Rayleigh scattering. Shorter blue wavelengths scatter more than red ones.
BLEU-4 Score: 0.0043

Prompt 3: How does grav

# DPO

In [25]:
import os
os.environ["WANDB_API_KEY"] = "946e923f7717c88464dc01b43cdcb664b74b23b6"
from trl import DPOConfig, DPOTrainer

Sft_model = AutoModelForCausalLM.from_pretrained('TinyLlama-new-1000', torch_dtype=torch.bfloat16).to(device)
tokenizer = AutoTokenizer.from_pretrained("TinyLlama-new-1000")
train_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")
train_dataset = train_dataset.select(range(500))


README.md:   0%|          | 0.00/643 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62135 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [26]:
Sft_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=2, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=2, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_

In [27]:
training_args = DPOConfig(
    per_device_train_batch_size=1,  # Reduce batch size
    remove_unused_columns=False,
    max_length=150,  # Reduce sequence length if possible
    max_prompt_length=256,
    bf16 = True,  # To maintain effective batch size
    dataloader_num_workers=0,
)


trainer = DPOTrainer(model=Sft_model, args=training_args, processing_class=tokenizer, train_dataset=train_dataset)
trainer.train()

Extracting prompt in train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tahashamji45 (tahashamji45-institute-of-business-administration) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn